Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(response.content)

https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table 3')[0]

Loading in\data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 3']


In [4]:
observations = tab.excel_ref('B15').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [5]:
observations

{<L16 225.0>, <J19 6.4>, <K19 1.3>, <G20 96.0>, <I15 5256.0>, <M20 89.5>, <H20 76.3>, <B21 '*'>, <D15 294.0>, <E16 '-'>, <B18 '*'>, <M21 3.5>, <G17 1643.0>, <I20 88.6>, <L19 16.6>, <H17 810.0>, <J17 2297.0>, <E19 '-'>, <K15 2036.0>, <M15 5969.0>, <J16 165.0>, <D17 270.0>, <K20 96.5>, <E21 '*'>, <H21 '*'>, <I17 4656.0>, <H19 '*'>, <D20 91.8>, <C15 324.0>, <K18 45.0>, <J21 4.5>, <J15 2577.0>, <L17 1080.0>, <G21 '*'>, <I18 '*'>, <G19 '*'>, <E17 685.0>, <G18 '*'>, <K21 2.2>, <H18 '*'>, <I19 '*'>, <F15 2482.0>, <H16 '*'>, <L21 3.8>, <B15 95.0>, <H15 1062.0>, <E15 713.0>, <F18 '*'>, <B19 '-'>, <F19 '*'>, <F21 '*'>, <F17 2203.0>, <G16 '*'>, <M18 211.0>, <L18 51.0>, <I16 '*'>, <I21 '*'>, <C20 99.1>, <M17 5341.0>, <E18 '*'>, <C17 321.0>, <K16 27.0>, <E20 96.1>, <B16 '-'>, <F20 88.8>, <M16 417.0>, <B17 94.0>, <K17 1964.0>, <L15 1356.0>, <F16 '*'>, <J18 115.0>, <J20 89.1>, <L20 79.6>, <G15 1712.0>, <M19 7.0>, <B20 98.9>}

In [6]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A18 'Mixed'>, <A21 'Mixed (%)'>, <A17 'Non-residential '>, <A15 'Total'>, <A20 'Non-residential (%)'>, <A19 'Residential (%)'>, <A16 'Residential'>}

In [7]:
Treatment = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Treatment

{<C14 'Drugs Only'>, <H14 'Drugs & Alcohol'>, <I14 'Total'>, <F14 'Alcohol Only'>, <E14 'Total'>, <D14 'Drugs & Alcohol'>, <J14 'Alcohol Only'>, <G14 'Drugs Only'>, <L14 'Drugs & Alcohol'>, <B14 'Alcohol Only'>, <K14 'Drugs Only'>}

In [8]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<B13 'Under 18 '>, <M13 'Overall Total'>, <F13 '18 and over'>, <J13 'Treatment Type'>}

In [9]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Residential status',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Treatment Type,Residential status,Age,Measure Type,Unit
0,95,NaN,Alcohol Only,Total,Under 18,Count,People
1,324,NaN,Drugs Only,Total,Under 18,Count,People
2,294,NaN,Drugs & Alcohol,Total,Under 18,Count,People
3,713,NaN,Total,Total,Under 18,Count,People
4,2482,NaN,Alcohol Only,Total,18 and over,Count,People
5,1712,NaN,Drugs Only,Total,18 and over,Count,People
6,1062,NaN,Drugs & Alcohol,Total,18 and over,Count,People
7,5256,NaN,Total,Total,18 and over,Count,People
8,2577,NaN,Alcohol Only,Total,Treatment Type,Count,People
9,2036,NaN,Drugs Only,Total,Treatment Type,Count,People


In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table.dtypes

Value                 object
DATAMARKER            object
Treatment Type        object
Residential status    object
Age                   object
Measure Type          object
Unit                  object
dtype: object

In [14]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Residential status,Age,Measure Type,Unit
71,,*,Total,Mixed (%),18 and over,Count,People
72,4.5,NaN,Alcohol Only,Mixed (%),Treatment Type,Count,People
73,2.2,NaN,Drugs Only,Mixed (%),Treatment Type,Count,People
74,3.8,NaN,Drugs & Alcohol,Mixed (%),Treatment Type,Count,People
75,3.5,NaN,None,Mixed (%),Overall Total,Count,People


In [15]:
new_table.count()

Value                 76
DATAMARKER            24
Treatment Type        69
Residential status    76
Age                   76
Measure Type          76
Unit                  76
dtype: int64

In [16]:
new_table = new_table[new_table['Value'] !=  0 ]

In [17]:
new_table = new_table[new_table['Value'] !=  '' ]

In [18]:
new_table.count()

Value                 52
DATAMARKER             0
Treatment Type        45
Residential status    52
Age                   52
Measure Type          52
Unit                  52
dtype: int64

In [19]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [20]:
new_table['Treatment Type'].fillna('Total', inplace = True)

In [21]:
new_table = new_table[['Residential status','Age','Treatment Type','Measure Type','Value','Unit']]

In [22]:
new_table.head(5)

,Residential status,Age,Treatment Type,Measure Type,Value,Unit
0,Total,Under 18,Alcohol Only,Count,95,People
1,Total,Under 18,Drugs Only,Count,324,People
2,Total,Under 18,Drugs & Alcohol,Count,294,People
3,Total,Under 18,Total,Count,713,People
4,Total,18 and over,Alcohol Only,Count,2482,People


In [23]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('tab3.2.csv'), index = False)